In [1]:
import sys
sys.path.append('/home/mila/l/leo.gagnon/latent_control')

%load_ext autoreload
%autoreload 2
from lightning_modules.diffusion_prior import DiffusionPriorTask
import torch
import matplotlib.pyplot as plt
from data.diffusion import KnownLatentDiffusionDataset
from torch2jax import j2t, t2j

In [3]:
task = DiffusionPriorTask('3jswuy2j')
task.cuda();

Loaded dataset : (11288/1000)
Loaded checkpoing : last.ckpt
Loaded checkpoing : last.ckpt


In [4]:
seq = task.base_task.full_data.__getitems__([0])['input_ids']

/home/mila/l/leo.gagnon/latent_control/data/hmm.py:585: FutureWarning: None encountered in jnp.array(); this is currently treated as NaN. In the future this will result in an error.
  intv_envs = jnp.array(intv_envs)
/home/mila/l/leo.gagnon/latent_control/venv/lib/python3.10/site-packages/jax/_src/numpy/lax_numpy.py:4252: FutureWarning: None encountered in jnp.array(); this is currently treated as NaN. In the future this will result in an error.
  return array(a, dtype=dtype, copy=bool(copy), order=order, device=device)


In [5]:
dataset = task.train_data.dataset 
dataset: KnownLatentDiffusionDataset

In [34]:
cond_tokens = dataset.encode(seq).cuda()
cond_mask = torch.BoolTensor([True] * 190 + [False] * 10)[None].cuda()

In [35]:
z_t = task.diffusion_prior.sample(1, cond_tokens=cond_tokens, cond_mask=cond_mask)

sampling loop time step:   0%|          | 0/250 [00:00<?, ?it/s]

In [15]:
env_latents = j2t(task.base_task.full_data.index_to_latent)[0].to(torch.long).cuda()[None]
env_latents = task.base_task.model.encoder(true_latents=env_latents)

In [36]:
task.base_task.model.encoder.latent_embedding[1].weight @ z_t[0].T

tensor([[ -1.3200],
        [238.1345]], device='cuda:0')

In [19]:
task.base_task.model.encoder.latent_embedding[1].weight @ z_t[0].T

tensor([[189.8581],
        [ -6.5179]], device='cuda:0')

In [31]:
task.base_task.full_data.index_to_latent[0]

Array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int16)

In [33]:
[(latent_embds.weight @ z_t[0].T).argmax() for latent_embds in task.base_task.model.encoder.latent_embedding]

[tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(1, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(0, device='cuda:0'),
 tensor(1, device='cuda:0')]